# Selenium

In [ ]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = Driver(uc=True)
driver.get("https://www.frontiersin.org/articles/10.3389/frai.2022.1049584/full")
# print(res)
x = page_actions.get_text(driver, "div")

In [ ]:
print(x)

In [ ]:
print(x.split("Author keywords"))
print(len(x.split("Author keywords")))

In [ ]:
driver.quit()

# Scopus API

In [ ]:
from pybliometrics.scopus import AbstractRetrieval, SerialSearch, ScopusSearch
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import pprint
import re
from tqdm import tqdm
import time
from confluent_kafka import Producer, Consumer, KafkaError

Config

In [ ]:
api_key = '464f17417c90aff3a9e077e3952aece0'

headers = {
    'Accept':'application/json',
    'X-ELS-APIKey': '27594a4c1478f17ac42280535ab20312'
}

attributes = [
    "title",
    "abstract",
    "affiliation",
    "authors",
    "authkeywords",
    "publicationName",
    "publisher",
    "subject_areas",
    "subtypedescription",
    "citedby_count",
    "references",
    "issn"
]

In [ ]:
def get_paper_info(papers):
    all_papers = []
    count_none = 0
    for paper in tqdm(papers):
        data_dict = {}
        if paper["eid"]:
            ab = AbstractRetrieval(paper["eid"], view='FULL')
            for attr in attributes:
                if hasattr(ab, attr):
                    value = getattr(ab, attr)
                    if value is not None:
                        if attr == "affiliation":
                            uni_aff = []
                            country_aff = []
                            for aff in value:
                                uni_aff.append(aff.name)
                                country_aff.append(aff.country)
                            data_dict["University"] = list(set(uni_aff))
                            data_dict["Country"] = list(set(country_aff))
                        elif attr == "authors":
                            index_author = []
                            for author in value:
                                index_author.append(author.indexed_name)
                            data_dict["Index_authors"] = list(set(index_author))
                        elif attr == "subject_areas":
                            subject_area = []
                            subject_area_code = []
                            for subj in value:
                                subject_area.append(subj.area)
                                subject_area_code.append(subj.abbreviation)
                            data_dict["Subject_Area"] = list(set(subject_area))
                            data_dict["Subject_Area_Code"] = list(set(subject_area_code))
                        elif attr == "references":
                            data_dict["references_number"] = len(ab.references)
                            data_dict["references"] = ab.references
                        elif attr == "issn":
                            try:
                                pattern = r'\b\d{4}\b'
                                match = re.search(pattern, ab.abstract)
                                if match:
                                    year = match.group()
                                else:
                                    search = SerialSearch(query={"issn": value.split(" ")[0]})
                                    search_result = search.results[0]
                                    year = search_result.get("coverageStartYear")
                                data_dict["year"] = year
                            except:
                                print(paper)
                        else:
                            data_dict[attr] = value
                    else:
                        count_none += 1
                else:
                    data_dict[attr] = ""
        all_papers.append(data_dict)
    return all_papers, count_none

## For Test

In [ ]:
import datetime
current_year = datetime.datetime.now().year
year_digits = [int(digit) for digit in str(current_year)]
print("Current Year:", current_year)
print("Year Digits:", year_digits)

In [ ]:
current_year -= 6

In [ ]:
current_year

In [ ]:
years = [
    "2016,2018",
    # "2017,2019",
    # "2019,2021",
    # "2020,2022",
    # "2021,2023",
    # "2022,2024",
]
super_all_paper = []

for year in years:
    query = f'TITLE({title}) AND PUBYEAR > {year.split(",")[0]} AND PUBYEAR < {year.split(",")[1]}' if title else f'PUBYEAR > {year.split(",")[0]} AND PUBYEAR < {year.split(",")[1]}'
    url = f'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}&httpAccept=application%2Fjson&count=1'
    response = requests.get(url, headers)
    if response.status_code == 200:
        try:
            papers_from_search = json.loads(response.text)['search-results']['entry']
            paper_nec_data, _ = get_paper_info(papers_from_search)
            super_all_paper += paper_nec_data
        except:
            print(response.text)
    else:
        print("Error code", response.status_code)
        break

In [ ]:
super_all_paper

In [ ]:
references_title = []
for paper in super_all_paper:
    try:
        references = paper["references"]
        all_title = []
        for ref in references:
            all_title.append(ref.title)
        references_title.append(all_title)
    except:
        print("Error", paper)
        references_title.append(None)
    

In [ ]:
print(RED)

In [ ]:
len(super_all_paper)

In [ ]:
super_all_paper[0]

In [ ]:
type(json.dumps(super_all_paper))

In [ ]:
payload = {
    "api_key": "464f17417c90aff3a9e077e3952aece0"
}

In [ ]:
response = requests.post(url='http://localhost:8000/scopus_api', json=payload)

In [ ]:
response.text

In [ ]:
res = requests.post("http://localhost:8000/scopus_api", {
    api_key: "464f17417c90aff3a9e077e3952aece0"
})

In [ ]:
res.text

In [ ]:
import redis

rd = redis.Redis(
    host='localhost',
    port=6379,
    encoding="utf-8",
    decode_responses=True
    )

In [ ]:
connection = rd.ping()
print(connection)

In [ ]:
test = rd.get('paper:raw')

In [ ]:
test

In [ ]:
cursor, keys = rd.scan(cursor=0, match='paper:cu*')


In [ ]:
keys

In [ ]:
# cursor, keys = rd.scan(cursor=cursor, match='paper:*')

In [ ]:
cursor = 0
rd.set(1, "yay")
rd.get(1)
# cursor, keys = rd.scan(cursor=cursor, match='paper:*')
# while cursor > 0:
    # cursor, keys = rd.scan(cursor=cursor, match='username:*')  

# for key in keys:
#     print('found: ', key)

In [ ]:
super_all_paper_temp_json

In [ ]:
len(references_title)

In [ ]:
df = pd.DataFrame(super_all_paper)

In [ ]:
df["ref_title"] = references_title

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df_tmp = df.drop(["references"], axis=1)

In [ ]:
new_column_names = {
    'title': 'Title',
    'abstract': 'Abstract',
    'University': 'University',
    'Country': 'Country',
    'Index_authors': 'Index_authors',
    'authkeywords': 'Keywords',
    'publicationName': 'PublicationName',
    'publisher': 'Publisher',
    'Subject_Area': 'Subject_Area',
    'Subject_Area_Code': 'Subject_Area_CodeType',
    'subtypedescription': 'Citation_Type',
    'citedby_count': 'Citation_Number',
    'references_number': 'Reference_number',
    'year': 'Year',
    'ref_title': "Reference_title"
}

df_tmp.rename(columns=new_column_names, inplace=True)

In [ ]:
df_tmp["Reference_title"][0]

In [ ]:
df_tmp.columns

In [ ]:
df_tmp = pd.read_csv('data_phase2.csv')

In [ ]:
super_all_paper_temp_json = json.loads(df_tmp.to_json(orient = 'records'))

In [ ]:
len(super_all_paper_temp_json)

In [ ]:
super_all_paper_temp_json

In [ ]:
import datetime
import os

In [ ]:
import platform


In [ ]:
print("The hostname of this device is = ",platform.node())



In [ ]:
# Kafka configuration
kafka_broker = 'localhost:9093'
topic = 'test'

# Producer
producer_config = {
    'bootstrap.servers': kafka_broker
}

producer = Producer(producer_config)

# Function to send API request to Kafka
def send_api_request_to_kafka(paper, key):
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log = {
        "timestamp": now,
        "id": key,
        "device": platform.node(),
        "system": platform.system(),
        "action": f'Paper id: {key} has been added from user {platform.node()} in {platform.system()} at {now}',
    }
    producer.produce(topic, value=json.dumps(log))
    producer.flush()  # Make sure to flush to send the messages immediately

# Send API requests to Kafka
key = 1
for paper in tqdm(super_all_paper_temp_json):
    send_api_request_to_kafka(paper, key)
    key += 1


In [ ]:
# Consumer
consumer_config = {
    'bootstrap.servers': kafka_broker,
    'group.id': 'event_log_group',
    'auto.offset.reset': 'earliest',
}

consumer = Consumer(consumer_config)
consumer.subscribe([topic])

# Consumer to receive messages from Kafka
print('Running Consumer')
logs = []
count_null = 0

try:
    while count_null < 10:
        msg = consumer.poll(1.0)
        if msg is None:
            print("Message from consumer is None")
            count_null += 1
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print(KafkaError._PARTITION_EOF)
                continue
            else:
                print(msg.error())
                break
        else:
            log = json.loads(msg.value())
            print(log)
            logs.append(log)
except KeyboardInterrupt:
    pass
finally:
    consumer.close()

In [ ]:
logs

In [ ]:
json_data = json.dumps(logs)
df = pd.read_json(json_data)
df.to_csv("events_log.csv", index=False)

In [ ]:
len(my_super_all_paper_temp_con)

In [ ]:
df.shape

In [ ]:
import sys

In [ ]:
if sys.version_info >= (3, 6):
    import zipfile
else:
    import zipfile36 as zipfile

In [ ]:
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import from_json, col
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

In [ ]:
# schema = StructType([
#     StructField("Title", StringType(), True),
#     StructField("Abstract", StringType(), True),
#     StructField("University", StringType(), True),
#     StructField("Country", StringType(), True),
#     StructField("Index_authors", StringType(), True),
#     StructField("Keywords", StringType(), True),
#     StructField("PublicationName", StringType(), True),
#     StructField("Publisher", StringType(), True),
#     StructField("Subject_Area", StringType(), True),
#     StructField("Subject_Area_CodeType", StringType(), True),
#     StructField("Citation_Type", StringType(), True),
#     StructField("Citation_Number", IntegerType(), True),
#     StructField("Reference_number", StringType(), True),
#     StructField("Year", StringType(), True),
#     StructField("Reference_title", StringType(), True),
# ])

In [ ]:
# spark = SparkSession.builder.appName("ScopusDataProcessing").config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1").getOrCreate()
# kafka_df = (
#     spark.readStream
#     .format("kafka")
#     .option("kafka.bootstrap.servers", kafka_broker)
#     .option("subscribe", topic)
#     .option("subscribe", "devices")
#     .option("startingOffsets", "earliest")
#     .load()
# )


In [ ]:
schema

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("ScopusDataProcessing").getOrCreate()

# Define the schema for the incoming JSON messages
schema = StructType([StructField("title", StringType(), True),
                     StructField("authors", StringType(), True),
                     StructField("abstract", StringType(), True),
                     # Add more fields as needed
                     ])

# Define the Kafka consumer
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", kafka_broker)
    .option("subscribe", topic)
    .load()
)

# Parse the JSON data from Kafka
parsed_df = df.selectExpr("CAST(value AS STRING)").select(from_json("value", schema).alias("data")).select("data.*")

# Perform transformations, filtering, or aggregations
processed_df = parsed_df.filter(col("abstract").isNotNull())  # Example: Filter out records without abstract

# Define the query to write the processed data to an output sink (e.g., console)
query = (
    processed_df.writeStream
    .outputMode("append")
    .format("console")
    .start()
)

# Await termination of the query
query.awaitTermination()

In [ ]:
df_tmp.to_csv('data_phase1-1.csv', index=False)

In [ ]:
df_tmp["Reference_title"][0]

In [ ]:
# ref_filtered_list = [title != None for title in df_tmp["Reference_title"][0]]

filtered_titles = [title for title in df_tmp["Reference_title"][0] if title is not None]


In [ ]:
df_tmp['Clean_Reference_title'] = df_tmp['Reference_title'].apply(lambda titles: [title for title in df_tmp["Reference_title"] if title != None])

In [ ]:
df_tmp

In [ ]:
df1 = pd.read_csv('data_phase1-1.csv')

In [ ]:
df1.isnull().sum()

In [ ]:
df3 = pd.concat([df_tmp, df1])
df3.shape

In [ ]:
df3

In [ ]:
df3 = df3.drop_duplicates(subset=['Title'], keep='first')
df3

In [ ]:
df3["Title"].duplicated()

In [ ]:
df.to_csv("data_phase1.xlsv", index=False)

In [ ]:
df1 = pd.read_csv('data_phase1.csv')
df1

In [ ]:
df3 = pd.concat([df, df1])
# df3.drop_duplicates(subset=None, inplace=True)
df.shape

In [ ]:
df2 = pd.read_csv('data_phase2.csv')
df2

In [ ]:
df3 = pd.concat([df1, df2])
df3.drop_duplicates(subset=None, inplace=True)
df3

In [ ]:
df_tmp = pd.read_csv("data_phase1.csv")

In [ ]:
df_tmp

In [ ]:
df_tmp.isnull().sum()

In [ ]:
titles = df_tmp.Title.values
titles

In [ ]:
type(list(titles))

# For debugging

In [ ]:
response = requests.post("http://localhost:8000/scopus_api", headers)

In [ ]:
json.loads(response.text)

In [ ]:
response

In [ ]:
papers = json.loads(response.text)['search-results']['entry']
papers

In [ ]:
ab = AbstractRetrieval("2-s2.0-85171199378", view='FULL')
ab.subject_areas

In [ ]:
attributes = [
    "title",
    "abstract",
    "affiliation",
    "authors",
    "authkeywords",
    "publicationName",
    "publisher",
    "subject_areas",
    "subtypedescription",
    "citedby_count",
    "references",
    "issn"
]

In [ ]:
all_papers = []
count_none = 0

In [ ]:
for paper in papers:
    data_dict = {}
    if paper["eid"]:
        ab = AbstractRetrieval(paper["eid"], view='FULL')
        for attr in attributes:
            
            if hasattr(ab, attr):
                value = getattr(ab, attr)
                if value is not None:
                    if attr == "affiliation":
                        uni_aff = []
                        country_aff = []
                        for aff in value:
                            uni_aff.append(aff.name)
                            country_aff.append(aff.country)
                        data_dict["University"] = uni_aff
                        data_dict["Country"] = country_aff
                    elif attr == "authors":
                        index_author = []
                        for author in value:
                            index_author.append(author.indexed_name)
                        data_dict["Index_authors"] = index_author
                    elif attr == "subject_areas":
                        subject_area = []
                        subject_area_code = []
                        for subj in value:
                            subject_area.append(subj.area)
                            subject_area_code.append(subj.abbreviation)
                        data_dict["Subject_Area"] = subject_area
                        data_dict["Subject_Area_Code"] = subject_area_code
                    elif attr == "references":
                        data_dict["references_number"] = len(ab.references)
                    elif attr == "issn":
                        try:
                            pattern = r'\b\d{4}\b'
                            match = re.search(pattern, ab.abstract)
                            if match:
                                year = match.group()
                            else:
                                search = SerialSearch(query={"issn": value.split(" ")[0]})
                                search_result = search.results[0]
                                year = search_result.get("coverageStartYear")
                            data_dict["year"] = year
                        except:
                            print(paper)
                    else:
                        data_dict[attr] = value
                else:
                    count_none += 1
            else:
                data_dict[attr] = ""
    all_papers.append(data_dict)

In [ ]:
import re

text = "Elsevier © 2009 Elsevier B.V.The structural collapse of manganese-based materials during cycling greatly restricts its "

# Define the regex pattern
pattern = r'\b\d{4}\b'

# Use re.search to find the match in the text
match = re.search(pattern, text)

# Retrieve the matched year
if match:
    year = match.group()
    print(f"Year: {year}")
else:
    print("Year not found.")

In [ ]:
all_papers

In [ ]:
len(all_papers)

In [ ]:
engineering_mapping = {
    'CE': 'Civil Engineering',
    'ENV': 'Environmental Engineering',
    'BME': 'Biomedical Engineering',
    'PE': 'Petroleum Engineering',
    'METAL': 'Metallurgical Engineering',
    'ME': 'Mechanical Engineering',
    'EE': 'Electrical Engineering',
    'CPE': 'Computer Engineering',
    'OPTIC': 'Optical Engineering',
    'NANO': 'Nano Engineering',
    'CHE': 'Chemical Engineering',
    'MATENG': 'Materials Engineering',
    'AGRI': 'Agricultural Engineering',
    'IE': 'Industrial Engineering',
    'SAFETY': 'Safety Engineering',
    'MATH': 'Mathematics and Statistics',
    'MATSCI': 'Material Science'
}


In [ ]:
len(all_papers)

In [ ]:
df = pd.DataFrame(all_papers)

In [ ]:
df['Subject_Area2'] = df['Subject_Area'].copy()

In [ ]:
df['Subject_Area2'] = df['Subject_Area2'].apply(lambda codes: [engineering_mapping.get(code, 'Unknown') for code in codes])

In [ ]:
df


In [ ]:
# Display the DataFrame
df.to_csv("test.csv", index=False)

In [ ]:
for aff in ab.affiliation:
    print(aff.name)

In [ ]:
search = SerialSearch(query={"title": "Energy facilities: Management and design and technological innovations"})
search_result = search.results[0]

result = search_result

In [ ]:
result.get("coverageStartYear")

In [ ]:
print(pd.DataFrame(s.results).head())

In [ ]:
api_key = '27594a4c1478f17ac42280535ab20312'

In [ ]:
api_key = '27594a4c1478f17ac42280535ab20312'

headers = {
    'Accept':'application/json',
    'X-ELS-APIKey': '27594a4c1478f17ac42280535ab20312'
}

query = 'SUBJAREA(ENGI) AND PUBYEAR > 2018 AND PUBYEAR < 2023'

url = f'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}&httpAccept=application%2Fjson'


response = requests.get(url, headers)

# pprint.pprint(response.text)

# print(len(json.loads(response.text)['search-results']['entry']))
pprint.pprint(json.loads(response.text)['search-results']['entry'])